# Ransac and Outlier Removal

## Notebook Setup 
The following cell will install Drake, checkout the manipulation repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  

More details are available [here](http://manipulation.mit.edu/drake.html).

In [ ]:
import importlib
import os, sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='9099a8e3fcc5c48fe46f4374b3c6be46c11f2d90', drake_version='20200921', drake_build='nightly')

# python libraries
import numpy as np

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# TODO(russt): upstream this to drake
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw

import open3d as o3d 
import meshcat

from IPython.display import clear_output
clear_output()

from manipulation import FindResource

# Visualize Stanford Bunny 
pcd = o3d.io.read_point_cloud(FindResource("models/bunny/bun_zipper_res2.ply"))
pointcloud_model = np.asarray(pcd.points).transpose()

# First, clean the origin a bit to define nominal pose. 
X = np.array([[1., 0., 0., 0.0],
              [0., np.cos(np.pi/2), -np.sin(np.pi/2), 0.],
              [0., np.sin(np.pi/2), np.cos(np.pi/2), -0.05]])
Xtemp = RigidTransform(X)
X = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2), 0, 0.],
              [np.sin(np.pi/2), np.cos(np.pi/2), 0., 0.],
              [0., 0., 1., 0.]])
X = RigidTransform(X).multiply(Xtemp)

pointcloud_model = X.multiply(pointcloud_model)

# point clouds of planar surface
import numpy as np
grid_spec = 50
xy_axis = np.linspace(-0.5, 0.5, grid_spec)
plane_x, plane_y = np.meshgrid(xy_axis, xy_axis)
points_plane_xy = np.c_[plane_x.flatten(), plane_y.flatten(), np.zeros(grid_spec**2)]
bunny_w_plane = np.c_[points_plane_xy.T, pointcloud_model]

def fit_plane(xyzs):
    '''
    Args:
      xyzs is (N, 3) numpy array
    Returns:
      (4,) numpy array
    '''
    center = np.mean(xyzs, axis=0)
    cxyzs = xyzs - center
    U, S, V = np.linalg.svd(cxyzs)
    normal = V[-1]              # last row of V
    d = -center.dot(normal)
    plane_equation = np.hstack([normal, d])
    return plane_equation

# visualize a facet
def DrawFacet(vis, abcd, name, center=None,
              prefix='facets', radius=0.02, thickness=0.001, color=0xffffff, opacity=0.6):
    normal = np.array(abcd[:3]).astype(float)
    normal /= np.linalg.norm(normal)
    d = -abcd[3] / np.linalg.norm(normal)

    R = np.eye(3)
    R[:, 2] = normal
    z = normal
    if abs(z[0]) < 1e-8:
        x = np.array([0, -normal[2], normal[1]])
    else:
        x = np.array([-normal[1], normal[0], 0])
    x /= np.linalg.norm(x)
    R[:, 0] = x
    R[:, 1] = np.cross(z, x)

    X = np.eye(4)
    Rz = RollPitchYaw(np.pi/2, 0, 0).ToRotationMatrix().matrix()
    X[:3, :3] = R.dot(Rz)
    if center is None:
        X[:3, 3] = d * normal
    else:
        X[:3, 3] = center
              
    X_normal = X.copy()
    X_normal[:3, :3] = R
    
    material = meshcat.geometry.MeshLambertMaterial(
        color=color, opacity=opacity)
    
    vis[prefix][name]["plane"].set_object(
        meshcat.geometry.Cylinder(thickness, radius), material)
    
    normal_vertices = np.array([[0, 0, 0], [0, 0, radius]]).astype(float)
    vis[prefix][name]["normal"].set_object(
        meshcat.geometry.Line(meshcat.geometry.PointsGeometry(normal_vertices.T)))
    
    vis[prefix][name]["plane"].set_transform(X)
    vis[prefix][name]["normal"].set_transform(X_normal)

def generate_color_mat(color_vec, shape):
  color_mat = np.tile(np.array(color_vec).astype(np.float32).reshape(3,1), (1, shape[1]))
  return color_mat

vis = meshcat.Visualizer(zmq_url)
# vis = meshcat.Visualizer(zmq_url)
def visualize_point_clouds(pc_A, vis=None):
    if vis is None:
        vis = meshcat.Visualizer(zmq_url)
    vis["/Background"].set_property('visible', False)
    #vis["/Cameras/default/"].set_transform(tf.translation_matrix([0, 0, 1]))
    vis["/Cameras/default/rotated/<object>"].set_property("zoom", 10.5)

    vis["red_bunny"].set_object(g.PointCloud(pc_A, generate_color_mat([1, 0, 0], pc_A.shape), size=0.01))
    return vis

# Problem Description
In the lecture, we learned about the RANSAC algorithm. In this exercise, you will implement the RANSAC algorithm to separate the Stanford bunny from its environment!

**These are the main steps of the exercise:**
1. Implement the `ransac` method.
2. Implement the `remove_plane` method to remove the points that belong to the planar surface.

Let's first visualize the point clouds of Stanford bunny in meshcat!

In [ ]:
vis = visualize_point_clouds(bunny_w_plane, vis)

You should notice that now there is a planar surface underneath the bunny. You may assume the bunny is currently placed on a table, where the planar surface is the tabletop. In this exercise, your objective is to remove the planar surface.

A straightforward way to achieve a better fit is to remove the planar surface underneath the bunny. To do so, we provide you a function to fit a planar surface. 

Recall that a plane equation is of the form
$$a x + b y + c z + d = 0$$
where $[a,b,c]^T$ is a vector normal to the plane and (if it's a unit normal) $d$ is the negative of the distance from the origin to the plane in the direction of the normal.  We'll represent a plane by the vector $[a,b,c,d]$.

The fitted planes are shown as translucent disks of radius $r$ centered at the points. The gray lines represent the planes' normals.

In [ ]:
plane_equation = fit_plane(bunny_w_plane.T)
print(plane_equation)
DrawFacet(vis, plane_equation, 'naive_plane', center=[0,0,-plane_equation[-1]], thickness=0.005, radius=0.1)

You should notice that the planar surface cannot be fitted exactly either. This is because it takes account of all points in the scene to fit the plane. Since a significant portion of the point cloud belongs to the bunny, the fitted plane is noticeably elevated above the ground. 

To improve the result of the fitted plane, you will use RANSAC!

## RANSAC
With the presence of outliers (bunny), we can use RANSAC to get more reliable estimates. The idea is to fit a plane using many random choices of a minimal set of points (3), fit a plane for each one, count how many points are within a distance tolerance to that plane (the inliers), and return the estimate with the most inliers.

**Complete the function `ransac`.  It takes a data matrix, a tolerance, a value of iterations, and a model regressor. It returns an equation constructed by the model regressor and a count of inliers.**

In [ ]:
def ransac(point_cloud, model_fit_func, tolerance=1e-3, max_iterations=500):
    '''
    Args:
      point_cloud is (N, 3) numpy array
      tolerance is a float
      max_iterations is a (small) integer
    Returns:
      (4,) numpy array
    '''
    best_ic = 0                 # inlier count
    best_model = np.ones(4)     # plane equation ((4,) array)

    ##################
    # your code here
    ##################

    return  best_ic, best_model

Now you should have a lot better estimate of the planar surface with the use of RANSAC! Let's visualize the plane now!

In [ ]:
inlier_count, ransac_plane = ransac(bunny_w_plane.T, fit_plane, 0.001, 500)
print(ransac_plane)
DrawFacet(vis, ransac_plane, 'ransac_plane', center=[0,0,-ransac_plane[-1]], thickness=0.005, radius=0.1)

## Remove Planar Surface

Now all you need to do is to remove the points that belong to the planar surface. You may do so by rejecting all points that are 
\begin{equation}
|| a x + b y + c z + d || > tol
\end{equation}

Note that since you are fitting a plane, the bunny is this case is the "outlier". Your job, however, is to keep the bunny and remove the planar surface.

**Complete the function below to remove the points that belongs to the planar surface**.

In [ ]:
def remove_plane(point_cloud, ransac, tol=1e-4):
    """
    Find the nearest (Euclidean) neighbor in point_cloud_B for each
    point in point_cloud_A.
    Args:
        point_cloud: Nx3 numpy array of points
        ransac: The RANSAC function to use (call ransac(args))
        plane_equation: (4,) numpy array, contains the coefficients of the plane
    Returns:
        point_cloud_wo_plane: Nx3 numpy array of points
    """
    point_cloud_wo_plane = np.zeros((100,3))
    return point_cloud_wo_plane

In [ ]:
bunny_wo_plane = remove_plane(bunny_w_plane.T, ransac)
vis = visualize_point_clouds(bunny_wo_plane.T, vis)

## How will this notebook be Graded?##

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

For submission of this assignment, you must do two things. 
- Download and submit the notebook `ransac.ipynb` to Gradescope's notebook submission section, along with your notebook for the other problems.
- Copy and Paste your answer to the kinematic singularity problem to Gradescope's written submission section. 

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [4 pts] `ransac` must be implemented correctly. 
- [2 pts] `remove_plane` must be implemented correctly.

In [ ]:
from manipulation.exercises.pose.test_ransac import TestRANSAC
from manipulation.exercises.grader import Grader 

Grader.grade_output([TestRANSAC], [locals()], 'results.json')
Grader.print_test_results('results.json')